### **แนวทางการจัดการ flow ของ Realtime Streaming**

##### 1️⃣ Control Rate ของ Data Input (Kafka / Producer)

- Throttle Producer: อย่าให้ producer ยิงข้อมูลเร็วเกินไป ใช้ `time.sleep()` หรือ queue mechanism

- Batch ข้อมูลก่อนส่ง: ถ้ามีข้อมูลจำนวนมากในช่วงสั้น ๆ ให้รวมเป็น batch แล้วส่งทีเดียว ลดจำนวน write request

- ใช้ Kafka เป็น Buffer :
    
    - Kafka ช่วยรองรับ input ที่เร็วเกินไป
    
    - Spark จะอ่านจาก Kafka ตาม trigger ของมัน → database ไม่โดนกระหน่ำ

##### 2️⃣ Control Rate ของ Streaming Job (Spark Structured Streaming)

- 1.ใช้ Batch Size / Trigger Interval:
    
    - ปรับจำนวน record ที่ Spark จะดึงมาพร้อมกัน และช่วงเวลาในการเขียนลง Cassandra / Database

In [ ]:
# การเชื่อมต่อ Cassandra
streaming_query = (selection_df.writeStream
                   .format("org.apache.spark.sql.cassandra")
                   .option("checkpointLocation", "/tmp/checkpoint")
                   .option("keyspace", "spark_streams")
                   .option("table", "created_users")
                   .option("spark.cassandra.connection.host", "cassandra")
                   .option("spark.cassandra.connection.port", "9042")
                   .option("spark.cassandra.connection.local_dc", "datacenter1")
                   .trigger(processingTime="10 seconds")  # แทนรันต่อเนื่องทันที
                   .start())

- `processingTime="10 seconds"` → Spark จะสร้าง micro-batch ทุก 10 วินาที

- ข้อมูลที่เข้ามาระหว่าง batch จะถูกจัดเก็บใน Kafka จนถึง batch ถัดไป

- จะช่วยลดจำนวน write request ต่อวินาที → Cassandra / Database ไม่พัง

- 2.Backpressure / Rate 

    - จำกัดจำนวน offset ต่อ batch (optional, ป้องกัน batch ใหญ่เกินไป)

In [ ]:
# การอ่านข้อมูลจาก Kafka
spark_df = (spark_conn.readStream
            .format("kafka")
            .option("kafka.bootstrap.servers", "broker:29092")
            .option("subscribe", "users_created")
            .option("startingOffsets", "earliest")
            .option("maxOffsetsPerTrigger", 1000)  # อ่านทีละ 1000 records
            .load())

- ถ้า data ไหลเร็วมาก จะช่วยควบคุมว่า Spark จะไม่ดึงเยอะเกินไป → batch นึงจะดึงได้สูงสุด 1000 record

##### 3️⃣ Upsert / TTL ใน Cassandra

- ถ้าไม่จำเป็นต้องเก็บทุก record → ใช้ TTL (Time-to-Live)

- หรือใช้ upsert แทน insert แบบไม่ซ้ำ (key conflicts)

- ช่วยลด data accumulation

##### 4️⃣ Checkpoint & Restart Safety

- ตั้ง `checkpointLocation` เพื่อให้ Spark รู้ว่าข้อมูล batch ไหนถูกเขียนแล้ว

- สำคัญมากกับ Realtime Streaming → DB

- หาก Spark หยุด/Restart จะไม่เขียนซ้ำ

##### 5️⃣ Partitioning / Parallelism

- Cassandra รองรับ write ที่กระจายไปหลาย partition

- ปรับ Spark partition ให้ match Cassandra partition key → ลด hotspot

##### 💡 สรุป:

- อย่าให้ stream รันแบบ “รัว ๆ ทุก record” โดยตรง

- ตั้ง trigger interval + maxOffsetsPerTrigger

- ใช้ checkpoint + TTL ใน Cassandra

- ออกแบบ schema และ partition ให้เหมาะกับ high throughput

### **Data Quality Checks (DQC) หรือ Data Validation**

- เวลาทำ Realtime ETL แล้ว transform เลย เรื่อง EDA + data quality จะเปลี่ยนวิธีคิดนิดไม่เหมือน Batch

##### 🔹 Batch ETL + EDA (แบบปกติ)

- ดึงข้อมูลมาเยอะ ๆ ทีหลัง

- ทำ EDA เต็มรูปแบบ เช่นหาค่า missing, distribution, outlier, correlation

- เพราะมี dataset ก้อนใหญ่ให้วิเคราะห์

##### 🔹 Realtime ETL + Transform ทันที

- ไม่ค่อยมีโอกาสทำ EDA ทุกครั้ง เพราะ data ไหลเข้ามาเป็น stream

- สิ่งที่ทำแทนคือ Data Quality Checks (DQC) หรือ Data Validation แบบ real-time

### **รูปแบบ API ในการทำ Streaming**

#### 🔹 ปกติ API ที่เป็น Streaming มี 3 รูปแบบหลัก

**1. Long-lived HTTP connection (HTTP Streaming / SSE – Server-Sent Events)**

- คุณยิง request ไปครั้งเดียว → connection ไม่ปิด → server จะ push ข้อมูลใหม่มาเรื่อย ๆ

- เช่น Twitter Streaming API (เก่า), หรือพวก Stock market feed, IoT sensor data

- Python/Java สามารถเปิด connection ค้างไว้ แล้ว loop อ่านข้อมูลเป็น record ต่อ record

**2. WebSocket API**

- สร้าง 2-way connection → client/server ส่งข้อความหากันได้ตลอด

- นิยมกับ real-time เช่น Chat, Crypto price feed, Game server

- library อย่าง `websockets` (Python), หรือ `socket.io`

**3. Polling API (simulate streaming)**

- ไม่มี streaming ให้ → client ต้อง call REST API ซ้ำ ๆ (เช่นทุก 1 วิ/10 วิ)

- ได้ผลเหมือน “stream” แต่อยู่ในรูป micro-batch

- ตรงนี้แหละที่บางคนใช้ Airflow หรือ Cron jobs คุมรอบเวลา

#### 🔹 สรุป

- **REST API (ปกติ)** → ไม่มีการส่งข้อมูลต่อเนื่องมาเอง 
    - → ถ้าอยากได้ข้อมูล real-time ต้อง ทำ polling (ดึงเองเป็นรอบ ๆ) เช่น ทุก ๆ 1 นาที, 5 นาที 
    - → ตรงนี้แหละที่ Airflow เหมาะ เพราะ Airflow = scheduler ที่ดีสำหรับ batch/micro-batch

- **Streaming API (แท้ ๆ เช่น WebSocket, gRPC stream, SSE, MQTT)** → connect ไว้ข้อมูลจะ push มาเองแบบ sensor feed 
    - → ไม่ต้องผ่าน Airflow เพราะไม่ได้ออกแบบมา run loop แบบนี้ ยิงเข้ามาที่ kafka ได้เลย 
    - → ปกติใช้ Spark Structured Streaming, Flink, Kafka Consumer หรือ custom daemon script แทน

##### ✅ Pattern ที่เจอในงานจริง

**1. REST API + Airflow**

- Airflow operator ดึงข้อมูลทุก 1 นาที

- เก็บลง storage (เช่น Cassandra, BigQuery, S3, Postgres)

- เหมาะกับ micro-batch

**2. Streaming API + Spark/Kafka**

- Spark structured streaming ต่อ WebSocket → push เข้า Kafka → consumer เก็บลง DB

- เหมาะกับ real-time

**3. Hybrid**

- API ไม่มี streaming ให้ → ใช้ polling (Airflow หรือ Spark Structured Streaming + foreachBatch())

- API มี streaming ให้ → ใช้ Spark/Kafka, แต่ยัง schedule งาน downstream ด้วย Airflow